In [1]:
import pandas as pd
from spark_session import LocalSparkSession
from dataset import Dataset
from mr_id3 import MapReduceIDR3
from pyspark.mllib.tree import DecisionTree

In [2]:
from log import log
from sklearn.impute import SimpleImputer
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, sum
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

In [3]:
%time
num_fields = [
    'age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss',
    'hours_per_week', ]

categorical_fields = [
    'workclass', 'education',
    'marital_status', 'occupation', 'relationship',
    'race', 'sex', 'native_country', ]

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs


In [4]:
%time
target = 'label'
filename = 'dataset/adult_100x.data'

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs


In [5]:
%time
numbers_of_cores = list(range(1, 5))
dataset_sizes = [1, ]
metrics = []

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 10.3 µs


In [6]:
%time
for number_of_cores in numbers_of_cores:

    spark = LocalSparkSession(number_of_cores)
    spark.start()

    dataset = Dataset(spark.spark, filename, num_fields, categorical_fields, target)
    dataset.load()
    dataset.select_only_numerical_features()

    for dataset_size in dataset_sizes:
        df = dataset.multiply_dataset(dataset_size)

        mr_id3 = MapReduceIDR3(df)
        mr_id3.set_labeled_point()
        mr_id3.split()
        mr_id3.train()

        metric = mr_id3.get_metrics()
        metric['length_rows'] = df.count()
        metric['dataset_size'] = dataset_size
        metric['number_of_cores'] = number_of_cores
        metrics.append(metric)
        log(f"Metrics: Clusters {metric['number_of_cores']} - Dataset size {metric['dataset_size']}x - Time {metric['time']} seconds")

    spark.stop()

2022-10-16 12:30:32,931 [INFO] LocalSparkSession : Starting with 1 clusters


CPU times: user 3 µs, sys: 2 µs, total: 5 µs
Wall time: 9.06 µs


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/16 12:30:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/16 12:30:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/16 12:30:37 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/16 12:30:37 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/10/16 12:30:37 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


2022-10-16 12:30:38,943 [INFO] Dataset : Starting
2022-10-16 12:30:38,944 [INFO] Dataset : Loading Dataset dataset/adult_100x.data
2022-10-16 12:30:43,595 [INFO] Dataset : One Hot Encode Categorical Features
ERROR:root:Exception while sending command.                         (0 + 1) / 3]
Traceback (most recent call last):
  File "/Users/marcelovasconcellos/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=79>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/marcelovasconcellos/opt/anaconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/marcelovasconcellos/opt/anaconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkErr

Py4JError: An error occurred while calling o47.javaToPython

In [ ]:
%time
df = pd.DataFrame.from_dict(metrics)

In [ ]:
%time
df[['number_of_cores', 'time']].plot()

In [ ]:
df